In [44]:
import torch
import torch.nn as nn
from torch_cluster import knn_graph
from torch_geometric.nn import (EdgeConv, graclus, max_pool, max_pool_x,
                                global_mean_pool, global_max_pool,
                                global_add_pool,BatchNorm)
from torch_geometric.utils import normalized_cut, degree
from torch_geometric.utils.undirected import to_undirected

x = torch.tensor([[-1., -1.], [-1., 1.], [1., -1.], [1., 1.], [0, 0]])
batch = torch.tensor([0, 0, 0, 0, 0])

k = 2

edgeconv1 = EdgeConv(nn=nn.Linear(2*2, 2), aggr="sum")

In [26]:
def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

In [27]:
x

tensor([[-1., -1.],
        [-1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 0.,  0.]])

In [38]:
edge_index = to_undirected(knn_graph(x, k, batch))
edge_index

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4],
        [1, 2, 4, 0, 3, 4, 0, 4, 1, 4, 0, 1, 2, 3]])

In [29]:
x_edgeconv1 = edgeconv1(x, edge_index)
x_edgeconv1

tensor([[-1.8514, -1.1597],
        [ 0.7960, -1.9523],
        [ 0.4476,  0.7679],
        [ 1.6644,  0.9080],
        [ 0.5838, -0.4750]], grad_fn=<ScatterAddBackward0>)

In [30]:
weight = normalized_cut_2d(edge_index, x_edgeconv1)
weight

tensor([1.8423, 2.5001, 1.4756, 1.8423, 2.4911, 0.8706, 2.5001, 0.9377, 2.4911,
        1.3164, 1.4756, 0.8706, 0.9377, 1.3164], grad_fn=<MulBackward0>)

In [33]:
row, col = edge_index
pos = x_edgeconv1
torch.norm(pos[row] - pos[col], p=2, dim=1)

tensor([2.7635, 3.0001, 2.5296, 2.7635, 2.9893, 1.4925, 3.0001, 1.2503, 2.9893,
        1.7552, 2.5296, 1.4925, 1.2503, 1.7552],
       grad_fn=<LinalgVectorNormBackward0>)

In [43]:
weight = normalized_cut(edge_index, torch.norm(pos[row] - pos[col], p=2, dim=1), num_nodes=pos.size(0))
weight

tensor([1.8423, 2.5001, 1.4756, 1.8423, 2.4911, 0.8706, 2.5001, 0.9377, 2.4911,
        1.3164, 1.4756, 0.8706, 0.9377, 1.3164], grad_fn=<MulBackward0>)

In [46]:
cluster = graclus(edge_index, weight, x_edgeconv1.size(0))
cluster

tensor([0, 1, 0, 3, 3])